Libraries to inport

In [5]:
from win32com.client import Dispatch, GetActiveObject
from win32com.shell import shell, shellcon
from pathlib import Path
from datetime import date
from shutil import make_archive
import time
import os
import re
import creds

Access the desktop folder

In [6]:
USSER_DESKTOP_PATH = os.path.join(os.environ['USERPROFILE'], 'Desktop')
CSIDL_DESKTOP = shellcon.CSIDL_DESKTOP
USSER_DESKTOP_CSIDL = shell.SHGetFolderPath(0, CSIDL_DESKTOP, None, 0)

# Emails
USER_EMAIL = creds.USER_EMAIL
WMSERVICE_EMAIL = creds.WMSERVICE_EMAIL
XAASIT_EMAIL = creds.XAASIT_EMAIL
# Email sub folders
CHECK_ERROR = '1_Check_Error'
TEST_XAASIT = 'Test Xaas'


In [7]:
print(type(USSER_DESKTOP_PATH))
print(USSER_DESKTOP_PATH)
print(type(USSER_DESKTOP_CSIDL))
print(USSER_DESKTOP_CSIDL)

<class 'str'>
C:\Users\Anticus\Desktop
<class 'str'>
C:\Users\Anticus\Desktop


In [72]:
def email_connection(user_email_address:str, sub_folder:str, main_folder = 'Inbox'):
    # Connect to email
    OUTLOOK = Dispatch('Outlook.Application')
    for i in range(10): # Try to connect to Outlook. Thie loop is needed because of the: "AttributeError - Outlook.Application.GetNameSpace" thet some times appears
        try:
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        except Exception as outlook_NameSpace_err:
            print(f'### Erron on run: {i} -- {outlook_NameSpace_err}')
            time.sleep(1)
        else:
            break

    
    for folder in OUTLOOK_NameSpace.Folders:
        print(folder)
    print('------------------------')
    print(OUTLOOK_NameSpace.Folders[3])
    
    addrBook = OUTLOOK_NameSpace.Folders.GetPrevious()
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)
    print(f'FOLDER: {OUTLOOK_NameSpace.Folders('Public Folders - xaasit@profidata.com')}')
    print(f'FOLDER: {OUTLOOK_NameSpace.Folders('xaasit@profidata.com')}')

    print(dir(OUTLOOK_NameSpace.Folders))

    # Get the emails
    emails = FOLDER.Items
    # Sort messages by ReceivedTime (descending order for most recent)
    emails.Sort('[ReceivedTime]', 1) # xlAscending = 1 # xlDescending = 2

    return OUTLOOK, emails

OUTLOOK, Test_XaasIT_emails = email_connection(USER_EMAIL, TEST_XAASIT)
# OUTLOOK, XaasIT_emails = email_connection(XAASIT_EMAIL, CHECK_ERROR)



Iftaas (PDM)
WMService
Public Folders - xaasit@profidata.com
xaas-it (PDM)
xaas (PDM)
xaasit@profidata.com
Andrei.Paveliuc@profidata.com
assetview-support
Public Folders - Andrei.Paveliuc@profidata.com
------------------------
xaas-it (PDM)
FOLDER: Public Folders - xaasit@profidata.com
FOLDER: xaasit@profidata.com
['Add', 'AddRef', 'Application', 'Class', 'Count', 'GetFirst', 'GetIDsOfNames', 'GetLast', 'GetNext', 'GetPrevious', 'GetTypeInfo', 'GetTypeInfoCount', 'Invoke', 'Item', 'Parent', 'QueryInterface', 'RawTable', 'Release', 'Remove', 'Session', '_ApplyTypes_', '_FlagAsMethod', '_LazyAddAttr_', '_NewEnum', '_Release_', '_UpdateWithITypeInfo_', '__AttrToID__', '__LazyMap__', '__bool__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__int__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__n

In [ ]:
print(OUTLOOK_NameSpace.Accounts[1])
print(OUTLOOK_NameSpace.Folders[3])

    for folder in OUTLOOK_NameSpace.Folders:
        print(folder)
    print('------------------------')
    print(OUTLOOK_NameSpace.Folders[3])
    # print(dir(OUTLOOK_NameSpace.Accounts))
    addrBook = OUTLOOK_NameSpace.Folders.GetPrevious()
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)
    print(f'FOLDER: {OUTLOOK_NameSpace.Folders('Public Folders - xaasit@profidata.com')}')
    print(f'FOLDER: {OUTLOOK_NameSpace.Folders('xaasit@profidata.com').Count}')

In [12]:
def email_connection(user_email_address:str, sub_folder:str, main_folder = 'Inbox'):
    # Connect to email
    for i in range(10): # Try to connect to Outlook. Thie loop is needed because of the: "AttributeError - Outlook.Application.GetNameSpace" thet some times appears
        try:
            OUTLOOK = Dispatch('Outlook.Application')
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        except AttributeError as outlook_NameSpace_err:
            print(f'### Erron on run: {i} -- {outlook_NameSpace_err}')
        print(i)
    # Connect to sub email folder
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)

    # Get the emails
    emails = FOLDER.Items
    # Sort messages by ReceivedTime (descending order for most recent)
    emails.Sort('[ReceivedTime]', 1) # xlAscending = 1 # xlDescending = 2

    return OUTLOOK, emails


OUTLOOK, Test_XaasIT_emails = email_connection(USER_EMAIL, TEST_XAASIT)
# OUTLOOK, XaasIT_emails = email_connection(XAASIT_EMAIL, CHECK_ERROR)

0
1
2
3
4
5
6
7
8
9


In [ ]:
def email_connection(user_email_address:str, sub_folder:str, main_folder = 'Inbox'):
    # Connect to email
    for i in range(10): # Try to connect to Outlook. Thie loop is needed because of the: "AttributeError - Outlook.Application.GetNameSpace" thet some times appears
        try:
            OUTLOOK = Dispatch('Outlook.Application')
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        except AttributeError as outlook_NameSpace_err:
            print(f'### Erron on run: {i} -- {outlook_NameSpace_err}')
        except Exception:
            if not is_outlook_running():
                open_outlook()
            OUTLOOK = Dispatch('Outlook.Application')
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        print(i)
    # Connect to sub email folder
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)

    # Get the emails
    emails = FOLDER.Items
    # Sort messages by ReceivedTime (descending order for most recent)
    emails.Sort('[ReceivedTime]', 1) # xlAscending = 1 # xlDescending = 2

    return OUTLOOK, emails

OUTLOOK, Test_XaasIT_emails = email_connection(USER_EMAIL, TEST_XAASIT)
# OUTLOOK, XaasIT_emails = email_connection(XAASIT_EMAIL, CHECK_ERROR)

In [ ]:
    print(OUTLOOK_NameSpace.Folders.Count)
    addrBook = OUTLOOK_NameSpace.Folders(user_email_address).Folders.GetFirst()
    print(addrBook.Name)
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)

In [ ]:
    print(OUTLOOK_NameSpace.Folders.Count)
    addrBook = OUTLOOK_NameSpace.Folders.GetFirst()
    print(addrBook.Name)
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)

In [ ]:
# 1 - C# tot python
# 2 - Ai folosit paranteze pentru proptirtatii
# 3 - Diferenta dintre _Folders si Folders
# 4 - Git